In [1]:
import json
import pandas
import pyarrow
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [28]:
test_query = """
    select
        date_trunc(metrc_sales_receipts.sales_datetime, MONTH) as date_type,
        metrc_sales_transactions.product_category_name,
        sum(metrc_sales_transactions.total_price)
    from
        companies
        inner join metrc_sales_receipts on metrc_sales_receipts.company_id = metrc_sales_receipts.company_id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'RA'
        and metrc_sales_receipts.sales_datetime >= '2020-01-01'
    group by
        date_trunc(metrc_sales_receipts.sales_datetime, MONTH),
        metrc_sales_transactions.product_category_name
    order by
        date_type asc,
        product_category_name asc
"""

incoming_transfer_packages_query = """
    select
        metrc_transfers.id as transfer_row_id,
        metrc_deliveries.id as delivery_row_id,
        metrc_transfer_packages.id as package_row_id,
        case
            when company_deliveries.delivery_type = 'INCOMING_UNKNOWN' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'OUTGOING_UNKNOWN' then 'OUTGOING_TO_PAYOR'
            when company_deliveries.delivery_type = 'OUTGOING_TO_PAYOR' then 'OUTGOING_TO_PAYOR'
            else company_deliveries.delivery_type
        end as delivery_type,
        company_deliveries.license_number,
        metrc_transfers.manifest_number,
        metrc_transfers.created_date,
        metrc_deliveries.received_datetime,
        metrc_transfers.shipper_facility_license_number,
        metrc_transfers.shipper_facility_name,
        metrc_deliveries.recipient_facility_license_number,
        metrc_deliveries.recipient_facility_name,
        metrc_deliveries.shipment_type_name,
        metrc_deliveries.shipment_transaction_type,
        metrc_transfer_packages.package_id,
        metrc_transfer_packages.package_label,
        metrc_transfer_packages.type,
        metrc_transfer_packages.package_payload.shipmentpackagestate as shipment_package_state,
        metrc_transfer_packages.package_payload.istestingsample as is_testing_sample,
        metrc_transfer_packages.package_payload.istradesample as is_trade_sample,
        metrc_transfer_packages.product_category_name,
        metrc_transfer_packages.product_name,
        metrc_transfer_packages.lab_results_status as package_lab_results_status,
        metrc_transfer_packages.shipper_wholesale_price,
        metrc_transfer_packages.shipped_quantity,
        metrc_transfer_packages.shipped_unit_of_measure,
        metrc_transfer_packages.received_quantity,
        metrc_transfer_packages.received_unit_of_measure,
        metrc_transfer_packages.package_payload.itemunitweight as item_unit_weight,
        metrc_transfer_packages.package_payload.itemunitweightunitofmeasurename as item_unit_weight_unit_of_measure_name
    from
        metrc_transfers
        inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
        inner join companies on company_deliveries.company_id = companies.id
        inner join metrc_deliveries on metrc_transfers.id = metrc_deliveries.transfer_row_id
        inner join metrc_transfer_packages on metrc_deliveries.id = metrc_transfer_packages.delivery_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and (
            company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' or
            company_deliveries.delivery_type = 'INCOMING_UNKNOWN'
        )
        and metrc_transfers.created_date >= '2020-01-01'
    order by
        created_date desc
"""

sales_transactions_query = """
    select
        metrc_sales_receipts.id,
        metrc_sales_transactions.last_modified_at,
        metrc_sales_receipts.license_number,
        metrc_sales_receipts.receipt_id,
        metrc_sales_receipts.receipt_number,
        metrc_sales_receipts.type as receipt_type,
        metrc_sales_receipts.sales_customer_type,
        metrc_sales_receipts.sales_datetime,
        metrc_sales_receipts.total_packages,
        metrc_sales_receipts.total_price,
        metrc_sales_transactions.type as tx_type,
        metrc_sales_transactions.package_id as tx_package_id,
        metrc_sales_transactions.package_label as tx_package_label,
        metrc_sales_transactions.product_name as tx_product_name,
        metrc_sales_transactions.product_category_name as tx_product_category_name,
        metrc_sales_transactions.unit_of_measure as tx_unit_of_measure,
        metrc_sales_transactions.quantity_sold as tx_quantity_sold,
        metrc_sales_transactions.total_price as tx_total_price
    from
        metrc_sales_receipts
        inner join companies on metrc_sales_receipts.company_id = companies.id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and metrc_sales_receipts.sales_datetime >= '2021-01-01'
    order by
        metrc_sales_receipts.sales_datetime desc
"""

simple_query = """
SELECT COUNT(*) FROM `metrc_api_keys` LIMIT 1000
"""
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=BIGQUERY_CREDENTIALS_PATH)
test = pandas.read_sql_query(incoming_transfer_packages_query, engine)
#test = pandas.read_sql_query(sales_transactions_query, redshift_engine)

In [29]:
test[0:1000]

,transfer_row_id,delivery_row_id,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,a8424319-5f8b-4fa0-b1b0-3f91043dfaf5,d794b081-ec29-43d5-8ca0-ac0ef0f2527d,01f3adca-efa7-4619-98e7-fbc094078ac9,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002543985,2021-10-07,NaT,C11-0000056-LIC,"RUKLI, INC.",...,Vape Cartridge (weight - each),Strawberry Bananaz Cart .5g,passed,339.5,50.0,Each,NaN,None,0.5,Grams
1,a8424319-5f8b-4fa0-b1b0-3f91043dfaf5,d794b081-ec29-43d5-8ca0-ac0ef0f2527d,816aa732-e114-4bfa-a373-53b75d4b449f,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002543985,2021-10-07,NaT,C11-0000056-LIC,"RUKLI, INC.",...,Extract (weight - each),SJ - Black Jack 1g SJ-BJ01,passed,416.0,32.0,Each,NaN,None,1.0,Grams
2,a8424319-5f8b-4fa0-b1b0-3f91043dfaf5,d794b081-ec29-43d5-8ca0-ac0ef0f2527d,f509bc43-d277-4873-8d03-990ab60ebf85,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002543985,2021-10-07,NaT,C11-0000056-LIC,"RUKLI, INC.",...,Extract (weight - each),GMO 1g,passed,416.0,32.0,Each,NaN,None,1.0,Grams
3,a8424319-5f8b-4fa0-b1b0-3f91043dfaf5,d794b081-ec29-43d5-8ca0-ac0ef0f2527d,09e8b034-532a-4565-988d-a5ad588e64f4,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002543985,2021-10-07,NaT,C11-0000056-LIC,"RUKLI, INC.",...,Extract (weight - each),SJ - Z-Blend 1g SJ-ZB01,passed,416.0,32.0,Each,NaN,None,1.0,Grams
4,a8424319-5f8b-4fa0-b1b0-3f91043dfaf5,d794b081-ec29-43d5-8ca0-ac0ef0f2527d,200df930-534e-4c28-a40f-a752cb2de57b,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002543985,2021-10-07,NaT,C11-0000056-LIC,"RUKLI, INC.",...,Extract (weight - each),BW - Cloud 9 1g BW-C9-01,passed,352.0,32.0,Each,NaN,None,1.0,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65eb6f96-a1d8-463a-b4e1-84f370e751d9,bba0ed5f-7644-42af-94b7-bfbc0953202f,3ca3e301-85f4-4b5c-bddb-27072afdf120,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002462401,2021-09-17,2021-09-17 17:20:05+00:00,C11-0000349-LIC,MISSION HILLS PATIENTS COLLECTIVE,...,Flower (packaged gram - each),Cali Flwr Farm - SOFA King PURPS (1 GM Flower ...,passed,75.0,30.0,Each,30.0,Each,1.0,Grams
996,65eb6f96-a1d8-463a-b4e1-84f370e751d9,bba0ed5f-7644-42af-94b7-bfbc0953202f,8fe6efab-292c-4302-a7f2-a7fb51449b2d,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002462401,2021-09-17,2021-09-17 17:20:05+00:00,C11-0000349-LIC,MISSION HILLS PATIENTS COLLECTIVE,...,Flower (packaged gram - each),Rich & Ruthless - Purple Hulk (1 GM Flower (Ea...,passed,75.0,30.0,Each,30.0,Each,1.0,Grams
997,65eb6f96-a1d8-463a-b4e1-84f370e751d9,bba0ed5f-7644-42af-94b7-bfbc0953202f,76cd3a02-b0e3-4e56-927b-c79be0508ae4,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002462401,2021-09-17,2021-09-17 17:20:05+00:00,C11-0000349-LIC,MISSION HILLS PATIENTS COLLECTIVE,...,Flower (packaged gram - each),Sofa King - Sofa King SKUNKEE (1 GM Flower (Ea...,passed,75.0,30.0,Each,30.0,Each,1.0,Grams
998,65eb6f96-a1d8-463a-b4e1-84f370e751d9,bba0ed5f-7644-42af-94b7-bfbc0953202f,026c1400-ab4d-4992-aafe-be7eb821db5a,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002462401,2021-09-17,2021-09-17 17:20:05+00:00,C11-0000349-LIC,MISSION HILLS PATIENTS COLLECTIVE,...,Flower (packaged gram - each),Viva Sativa - Viva Fuego (1 GM Flower (Each)) ...,passed,75.0,30.0,Each,30.0,Each,1.0,Grams
